In [1]:
from pathlib import Path
import pathlib
import geopandas as gpd
from tval import trace_validator
import time

In [2]:
trace_validator.BaseValidator.set_snap_threshold_and_multiplier(0.001, 2)

In [3]:
gdfs = []

lidar_traces = [
    Path("dev/data_22062020/traces/SFin_500_integratedlineaments_korjattu150420.gpkg"),
                Path("dev/data_22062020/traces/Inkoo_200_integratedlineaments_korjattu140420.gpkg"),
                Path("dev/data_22062020/traces/Inkoo_Infinity_Lineaments.gpkg"),
               ]
lidar_areas = [
    Path("dev/data_22062020/areas/SFin_500_area.gpkg"), 
    Path("dev/data_22062020/areas/Inkoo_200_area_2020.gpkg"),
    Path("dev/data_22062020/areas/Infinity_Kopparnas.gpkg"), 
]

traces_20m = [
    #Path("dev/data_22062020/traces/Drone_all_korjattu150420.gpkg"), 
    Path("dev/data_22062020/traces/Detaildrone_all_170420_new.gpkg"), 
]

areas_20m = [
    Path("dev/data_22062020/areas/Drone_area_erased.gpkg"),
    Path("dev/data_22062020/areas/Detaildrone_area_merged_all.gpkg"),
]


for traces, area in zip(traces_20m, areas_20m):
    print(f"Validating {traces}")
    trace_gdf = gpd.read_file(traces)
    if trace_validator.BaseValidator.ERROR_COLUMN in trace_gdf.columns:
        print(f"Error column already in gdf. Old will be backed up in {traces}")
        trace_gdf[f"error_column_backup"] = trace_gdf[trace_validator.BaseValidator.ERROR_COLUMN]
        trace_gdf[trace_validator.BaseValidator.ERROR_COLUMN] = [
            [] for _ in trace_gdf.index
        ]
    area_gdf = gpd.read_file(area)
    for validator in (
    trace_validator.GeomTypeValidator
    , trace_validator.MultiJunctionValidator
    , trace_validator.VNodeValidator
    , trace_validator.MultipleCrosscutValidator
    , trace_validator.UnderlappingSnapValidator
    , trace_validator.TargetAreaSnapValidator
    ):
        print(f"Validating with {validator}")
        print(f"Start time is {time.asctime()}")
        try:
            trace_gdf = validator.execute(trace_gdf, area_gdf, auto_fix=True)
        except NotImplementedError:
            print(f"Cannot automatically fix {validator.ERROR}")
            trace_gdf = validator.execute(trace_gdf, area_gdf, auto_fix=False)
        gdfs.append(trace_gdf.copy())
    trace_gdf.astype(
        {trace_validator.BaseValidator.ERROR_COLUMN: 'str'}
    ).to_file(
        Path(f"dev/{traces.stem}_validated_01092020_2.gpkg"), driver="GPKG"
    )

Validating dev/data_22062020/traces/Detaildrone_all_170420_new.gpkg
Validating with <class 'tval.trace_validator.GeomTypeValidator'>
Start time is Tue Sep  1 21:37:56 2020
Unable to convert MultiLineString to LineString. MultiLineString probably consists of disjointed segments.
Unable to convert MultiLineString to LineString. MultiLineString probably consists of disjointed segments.
Unable to convert MultiLineString to LineString. MultiLineString probably consists of disjointed segments.
Validating with <class 'tval.trace_validator.MultiJunctionValidator'>
Start time is Tue Sep  1 21:37:57 2020
MultiJunctionValidator cannot automatically fix MULTI JUNCTION.
Validating with <class 'tval.trace_validator.VNodeValidator'>
Start time is Tue Sep  1 21:38:49 2020
VNodeValidator cannot automatically fix V NODE.
Validating with <class 'tval.trace_validator.MultipleCrosscutValidator'>
Start time is Tue Sep  1 21:39:23 2020
MultipleCrosscutValidator cannot automatically fix MULTIPLE CROSSCUTS.
Va